# 卷积神经网络例子

用tensorflow创建一个卷积神经网络

·作者：朱海

·项目：https://github.com/Demohai/my_tensorflow_learn/tree/master/Neural_Network/convolutional_network

In [14]:
import tensorflow as tf

In [15]:
# 导入MNIST数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data_set/", one_hot=True)

Extracting ./MNIST_data_set/train-images-idx3-ubyte.gz
Extracting ./MNIST_data_set/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data_set/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data_set/t10k-labels-idx1-ubyte.gz


In [16]:
print("The shape of train set x is:", mnist.train.images.shape)
print("The shape of train set y is:", mnist.train.labels.shape)
print("The shape of validation set x is:", mnist.validation.images.shape)
print("The shape of validation set y is:", mnist.validation.labels.shape)
print("The shape of test set x is:", mnist.test.images.shape)
print("The shape of test set y is:", mnist.test.labels.shape)

The shape of train set x is: (55000, 784)
The shape of train set y is: (55000, 10)
The shape of validation set x is: (5000, 784)
The shape of validation set y is: (5000, 10)
The shape of test set x is: (10000, 784)
The shape of test set y is: (10000, 10)


In [17]:
# 训练参数
learning_rate = 0.001
num_steps = 200
batch_size = 128

In [18]:
# 神经网络参数
num_input = 784   #MNist数据输入为28*28 = 784
num_class = 10  #手写字母共有九个(0-9)

In [19]:
# 定义输入函数
def data_input(num_input, num_class):
    x = tf.placeholder(tf.float32, shape=[None, num_input])
    y = tf.placeholder(tf.float32, shape=[None, num_class])
    keep_prob = tf.placeholder(tf.float32)
    return x, y, keep_prob

In [20]:
# 创建简单的卷积层
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.add(x, b)       
    return tf.nn.relu(x)

In [21]:
# 创建简单的池化层
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [22]:
# 参数W和b初始化函数
def parameters_initialize():
    # filter:5x5, 1 input, 32 outputs
    WC1 = tf.Variable(tf.random_normal([5, 5, 1, 32])) #随机正态分布
    bc1 = tf.Variable(tf.random_normal([32]))
    # filter:5x5, 32 inputs, 64 outputs
    WC2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
    bc2 = tf.Variable(tf.random_normal([64]))
    # fully connected: 7*7*64 inputs, 1024 outputs
    WD1 = tf.Variable(tf.random_normal([7*7*64, 1024])) #经过两个padding='SAME'的卷积层，和两个(2, 2)的MaxPool后，输出变为(7,7,64)
    bd1 = tf.Variable(tf.random_normal([1024]))
    # fully connected: 1024 inputs, 10 outputs
    WD2 = tf.Variable(tf.random_normal([1024, 10]))
    bd2 = tf.Variable(tf.random_normal([10]))
    
    weights = {"WC1": WC1, "WC2": WC2, "WD1": WD1, "WD2": WD2}
    bias = {"bc1": bc1, "bc2": bc2, "bd1": bd1, "bd2": bd2}
    return weights, bias

In [23]:
# 构建卷积神经网络模型
def conv_net(x, weights, bias, keep_prob):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # Convolution Layer1
    WC1 = weights["WC1"]
    bc1 = bias["bc1"]
    conv1 = conv2d(x, WC1, bc1)
    conv1 = maxpool2d(conv1, k=2)
    
    # Convolution Layer2
    WC2 = weights["WC2"]
    bc2 = bias["bc2"]
    conv2 = conv2d(conv1, WC2, bc2)
    conv2 = maxpool2d(conv2, k=2)
    
    # Fully Connected Layer1
    # Reshape conv2 output to fit fully connected layer input
    WD1 = weights["WD1"]
    bd1 = bias["bd1"]
    fc1 = tf.reshape(conv2, [-1, WD1.get_shape().as_list()[0]])
    
    fc1 = tf.add(tf.matmul(fc1, WD1), bd1)  # 使用tf.matmul时，注意两个矩阵相乘的顺序，否则会造成输出矩阵形状有误
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    # Fully Connected Layer2(Output)
    WD2 = weights["WD2"]
    bd2 = bias["bd2"]
    fc2 = tf.add(tf.matmul(fc1, WD2), bd2)
    
    
    return fc2

In [24]:
def model(num_input=784, num_class=10, learning_rate=0.001, num_steps=200, batch_size=128):
    x, y, keep_prob = data_input(num_input, num_class)
    weights, bias = parameters_initialize()
    fc2 = conv_net(x, weights, bias, keep_prob)
    y_pred = tf.nn.softmax(fc2)
    
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc2, labels=y)) 
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    
    # Start training
    with tf.Session() as sess:
        # Run the initializer
        sess.run(init)
        
        for i in range(num_steps):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            if i % 10 == 0:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
                print("Step " + str(i) + ", Minibatch Loss= " + \
                    "{:.4f}".format(loss) + ", Training Accuracy= " + \
                    "{:.3f}".format(acc))
            # Run optimization op(backprop)
            sess.run(train_op, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.8})
        print("Optimization finished")
        
        # Calculate accuracy for  MNIST test images
        print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

In [25]:
model()

Step 0, Minibatch Loss= 154981.9375, Training Accuracy= 0.125
Step 10, Minibatch Loss= 41111.5664, Training Accuracy= 0.195
Step 20, Minibatch Loss= 15696.7266, Training Accuracy= 0.414
Step 30, Minibatch Loss= 10589.9141, Training Accuracy= 0.555
Step 40, Minibatch Loss= 6577.9272, Training Accuracy= 0.734
Step 50, Minibatch Loss= 4819.0166, Training Accuracy= 0.805
Step 60, Minibatch Loss= 5597.4814, Training Accuracy= 0.711
Step 70, Minibatch Loss= 2081.1616, Training Accuracy= 0.852
Step 80, Minibatch Loss= 3917.0784, Training Accuracy= 0.805
Step 90, Minibatch Loss= 3314.1797, Training Accuracy= 0.797
Step 100, Minibatch Loss= 1136.6399, Training Accuracy= 0.898
Step 110, Minibatch Loss= 2390.5046, Training Accuracy= 0.914
Step 120, Minibatch Loss= 2496.7153, Training Accuracy= 0.859
Step 130, Minibatch Loss= 2777.8389, Training Accuracy= 0.859
Step 140, Minibatch Loss= 2528.3164, Training Accuracy= 0.883
Step 150, Minibatch Loss= 1969.7141, Training Accuracy= 0.906
Step 160, Mini